<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/18_Habitat_heterogeneity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculate habitat heterogeneity metrics for NDVI using Sentinel imagery

See [here](https://developers.google.com/earth-engine/guides/python_install#syntax) for differences between Javascript and Python syntax

Main [tutorial](https://courses.spatialthoughts.com/end-to-end-gee.html#module-6-google-earth-engine-python-api) here

## Import gee and authenticate

In [ ]:
import ee
import re

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/LLBN")
aoi_name = "LLBN"


# Indice
indice = "NDVI"

# GEE layer ID
layer_dict = {
    indice + "_CV": "RS_054",
    indice + "_contrast": "RS_055",
    indice + "_diss": "RS_056",
    indice + "_ent": "RS_057",
    indice + "_idm": "RS_058"
}

# Date parameters (habitat metrics to be calculated based on averages over the last 5 years)
start_year = 2018
end_year = 2022

## Import Sentinel 2 image collection

Dataset starts in June 2015
Clouds can be mostly removed by using [COPERNICUS/S2_CLOUD_PROBABILITY](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_CLOUD_PROBABILITY). See [this tutorial](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless) explaining how to apply the cloud mask.



---


Sentinel-2 MSI: MultiSpectral Instrument, Level-1C
```ee.ImageCollection("COPERNICUS/S2")```



---


Harmonized Sentinel-2 MSI: MultiSpectral Instrument, Level-2A
```ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")```



---


Sentinel-2 MSI: MultiSpectral Instrument, Level-2A
```ee.ImageCollection("COPERNICUS/S2_SR")```

In [ ]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

## NDVI processing

In [ ]:
def maskS2clouds(image):
  qa = image.select('QA60')
  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
  return image.updateMask(mask).divide(10000)

def addNDVI(image):
  imgb = image.select('B.*')
  ndvi = imgb.normalizedDifference(['B8','B4']).rename('NDVI')
  return image.addBands(ndvi)

start = ee.Date.fromYMD(start_year, 1, 1)
end = ee.Date.fromYMD(end_year, 12, 31)
date_range = ee.DateRange(start, end)

ndvi = ee.ImageCollection("COPERNICUS/S2") \
  .filterDate(date_range) \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
  .map(maskS2clouds) \
  .map(addNDVI) \
  .select('NDVI') \
  .mean() \
  .clip(aoi)

print(ndvi.getInfo())

In [ ]:
# Mask negative values (water, bare soil)
ndvi = ndvi.updateMask(ndvi.gte(0))

## Calculate second-order textture metrics

In [ ]:
glcm = ndvi \
  .unitScale(0, 1) \
  .multiply(1000) \
  .toInt() \
  .glcmTexture(size = 2)

In [ ]:
# Checks
print(glcm.getInfo())
print(glcm.bandNames().getInfo())

band_names = glcm.bandNames().getInfo()
pattern = r"_contrast|_diss|_ent|_idm"
filtered_band_names = [name for name in band_names if re.search(pattern, name)]
print(filtered_band_names)

texture_metrics = glcm.select(filtered_band_names)
print(texture_metrics.getInfo())

## Calculate CV

In [ ]:
# Define a neighborhood kernel to compute statistics on neighboring pixels
kernelSize = 2  # Adjust the kernel size as needed
kernel = ee.Kernel.square(kernelSize)

# Compute the mean and standard deviation of the NDVI values within each pixel's neighborhood
meanImage = ndvi \
  .reduceNeighborhood(reducer=ee.Reducer.mean(), kernel=kernel)

stdDevImage = ndvi \
  .reduceNeighborhood(reducer=ee.Reducer.stdDev(), kernel=kernel)

# Calculate the coefficient of variation (CV) using the computed mean and standard deviation
cvImage = stdDevImage.divide(meanImage).multiply(100)  # Multiply by 100 for percentage representation

## Export data

In [ ]:
for i in texture_metrics.bandNames().getInfo():
  print(layer_dict[i])
  output_img = texture_metrics.select(i)
  output_name = f"{layer_dict[i]}_{aoi_name}"
  task = ee.batch.Export.image.toDrive(image = output_img,
                                      region = aoi.geometry(),
                                      description = "EXPORT IMAGE TO DRIVE",
                                      folder = "GEE_exports",
                                      fileNamePrefix = output_name,
                                      scale = 30,
                                      maxPixels = 10e12,
                                      crs = "EPSG:4326"
                                      )
  task.start()
  print("STARTED TASK ", i)

In [ ]:
layer_id = layer_dict[indice + "_CV"]
output_name = f"{layer_id}_{aoi_name}"

task = ee.batch.Export.image.toDrive(image = cvImage,
                                      region = aoi.geometry(),
                                      description = "EXPORT IMAGE TO DRIVE",
                                      folder = "GEE_exports",
                                      fileNamePrefix = output_name,
                                      scale = 30,
                                      maxPixels = 10e12,
                                      crs = "EPSG:4326"
                                      )
task.start()

Layers below just needed for checks, not for inclusion into AGOL

In [ ]:
# task = ee.batch.Export.image.toDrive(image = ndvi,
#                                       region = aoi.geometry(),
#                                       description = "EXPORT IMAGE TO DRIVE",
#                                       folder = "GEE_exports",
#                                       fileNamePrefix = "ndvi",
#                                       scale = 30,
#                                       maxPixels = 10e12
#                                       )
# task.start()

In [ ]:
# task = ee.batch.Export.image.toDrive(image = meanImage,
#                                       region = aoi.geometry(),
#                                       description = "EXPORT IMAGE TO DRIVE",
#                                       folder = "GEE_exports",
#                                       fileNamePrefix = "mean",
#                                       scale = 30,
#                                       maxPixels = 10e12
#                                       )
# task.start()

In [ ]:
# task = ee.batch.Export.image.toDrive(image = stdDevImage,
#                                       region = aoi.geometry(),
#                                       description = "EXPORT IMAGE TO DRIVE",
#                                       folder = "GEE_exports",
#                                       fileNamePrefix = "sd",
#                                       scale = 30,
#                                       maxPixels = 10e12
#                                       )
# task.start()

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
# tasks = ee.batch.Task.list()
# for task in tasks[0:ee.List.length(year_list).getInfo()]:
#   task_id = task.status()['id']
#   task_state = task.status()['state']
#   print(task_id, task_state)